
## Install required packages

Install the required packages and connect your SerpAPI account so agents can use it to retrieve news articles. Run the following commands on your terminal:

In [ ]:
%pip install composio-langchain
%pip install composio-core
%pip install langchain-community
%pip install huggingface_hub
%pip install google-search-results
%pip install python-dotenv

Use Composio to add SERP tool

In [ ]:
%composio login
%composio add serpapi

## Import base packages
Begin by importing the necessary packages:

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
#To use Open Source LLMs from huggingface
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chat_models import ChatHuggingFace
#Langchain Utilities for Agents
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain import hub
from langchain.tools.render import render_text_description
#Importing Composio
from composio_langchain import ComposioToolSet, Action, App
import os
import dotenv

## Initialise the Tools



In [ ]:
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(apps=[App.SERPAPI])

## Initialise the LLM

In [ ]:
dotenv.load_dotenv()
llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta", huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))
chat_model = ChatHuggingFace(llm=llm)

## Setup ReAct Style Prompt Template
Set up the prompt template using Langchain Hub:





In [ ]:
#We use Langchain Hub for the Prompt Template
prompt = hub.pull("hwchase17/react-json")
#prompt.partial assigns values to some parameters defined within the prompt template
prompt = prompt.partial(
tools=render_text_description(tools),
tool_names=", ".join([t.name for t in tools]),
)

## Define the Agent
Define the agent to process the news retrieval and summarization


In [ ]:
chat_model_with_stop = chat_model.bind(stop=["\nInvalidStop"])
agent = (
{
"input": lambda x: x["input"],
"agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
}
| prompt
| chat_model_with_stop
| ReActJsonSingleInputOutputParser()
)

## Executing the Agent
Use the agent to find the latest AI news and summarize it:


In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.return_intermediate_steps = True
res = agent_executor.invoke(
    {
        "input": "Use SERP to find the one latest AI news, take only description of article."
    }
)

res2 = agent_executor.invoke({
    "input": res['output']+' Summarize this'
})


## Putting it All Together

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace
from composio_langchain import ComposioToolSet, Action, App


llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta", huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

chat_model = ChatHuggingFace(llm=llm)
# Import from composio_langchain


# setup tools
composio_toolset = ComposioToolSet()
tools = composio_toolset.get_tools(apps=[App.SERPAPI])

# setup ReAct style prompt
prompt = hub.pull("hwchase17/react-json")
prompt = prompt.partial(
    tools=render_text_description(tools),
    tool_names=", ".join([t.name for t in tools]),
)

# define the agent
chat_model_with_stop = chat_model.bind(stop=["\nInvalidStop"])
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | chat_model_with_stop
    | ReActJsonSingleInputOutputParser()
)

# instantiate AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)
agent_executor.return_intermediate_steps = True
res = agent_executor.invoke(
    {
        "input": "Use SERP to find the one latest AI news, take only description of article."
    }
)

res2 = agent_executor.invoke({
    "input": res['output']+' Summarize this'
})

